In [5]:
import numpy as np

rands = np.random.rand(100)
print(rands)
print(np.max(rands))
print(np.min(rands))

[0.19284321 0.64483592 0.81964266 0.37520952 0.60914918 0.89227095
 0.68340758 0.16461045 0.51990035 0.05786046 0.7563089  0.31239104
 0.02200419 0.79603243 0.15704918 0.92408947 0.03748248 0.52815423
 0.29911559 0.14770247 0.95517907 0.06989029 0.24487559 0.96123882
 0.56430988 0.64793415 0.39907785 0.12374698 0.97084047 0.32695232
 0.19435574 0.18146556 0.60245782 0.38618425 0.40042983 0.76296885
 0.05294944 0.3390786  0.62075586 0.97489752 0.18641536 0.24099439
 0.47330193 0.75797819 0.18374187 0.4599742  0.51567137 0.9042155
 0.55189454 0.95377271 0.04002547 0.8288115  0.43933381 0.11689192
 0.28868133 0.89932802 0.27314228 0.35589719 0.22680227 0.66475185
 0.46892316 0.76414492 0.68976606 0.43206764 0.18595513 0.42586285
 0.13505824 0.10370549 0.27876298 0.70942789 0.24602899 0.63099955
 0.83644481 0.73922043 0.45935891 0.09504481 0.06336433 0.97592076
 0.92025828 0.12099717 0.97955773 0.85960098 0.98603625 0.54750236
 0.21376827 0.37269945 0.07969715 0.1797308  0.48670857 0.73243

In [1]:
import numpy as np
from scipy.stats import multivariate_normal

# 定义多元正态分布的参数
mean = np.array([0, 0])
cov = np.array([
    [1.0, 0.0],
    [0.0, 1.0]
])

# 创建一个分布对象
dist = multivariate_normal(mean=mean, cov=cov)

# 准备多个数据点
# 这些数据点应该是一个 (N, D) 的数组
# 其中 N=3 (3个点), D=2 (二维)
multiple_data_points = np.array([
    [0.0, 0.0],  # 第一个点
    [1.0, 0.0],  # 第二个点
    [0.0, 1.0]   # 第三个点
])

# 同时传入多个数据点
# x 参数接受 (N, D) 形状的数组
pdf_values = dist.pdf(x=multiple_data_points)

print("Mean:", mean)
print("Covariance:\n", cov)
print("Multiple data points:\n", multiple_data_points)
print("PDF values for multiple data points:", pdf_values)

# 验证一下，单独计算每个点的值，看是否一致
pdf_value_1 = dist.pdf(x=np.array([0.0, 0.0]))
pdf_value_2 = dist.pdf(x=np.array([1.0, 0.0]))
pdf_value_3 = dist.pdf(x=np.array([0.0, 1.0]))

print("\nVerification (individual calculations):")
print("[0, 0] PDF:", pdf_value_1)
print("[1, 0] PDF:", pdf_value_2)
print("[0, 1] PDF:", pdf_value_3)

# 它们应该与 pdf_values 数组中的值匹配

Mean: [0 0]
Covariance:
 [[1. 0.]
 [0. 1.]]
Multiple data points:
 [[0. 0.]
 [1. 0.]
 [0. 1.]]
PDF values for multiple data points: [0.15915494 0.09653235 0.09653235]

Verification (individual calculations):
[0, 0] PDF: 0.15915494309189535
[1, 0] PDF: 0.09653235263005393
[0, 1] PDF: 0.09653235263005393


In [4]:
from numpy.linalg import inv, det
def multivariate_normal_pdf_vectorized(x, means, cov):
    d = x.shape[0]
    cov_inv = inv(cov)
    cov_det = det(cov)
    norm_const = 1.0 / np.sqrt((2 * np.pi) ** d * cov_det)

    diffs = means - x  # shape: (N, D)
    exponents = -0.5 * np.sum(diffs @ cov_inv * diffs, axis=1)  # shape: (N,)
    return norm_const * np.exp(exponents)

In [5]:
import numpy as np
from scipy.stats import multivariate_normal

# --- 1. 定义一个 2D 的多元正态分布 (协方差固定) ---
# 假设这是我们的观测噪声协方差 R
observation_covariance_R = np.array([
    [0.01, 0.00],  # sigma_x^2 = 0.01
    [0.00, 0.01]   # sigma_y^2 = 0.01
])

# 创建一个 "冻结" 的分布对象，只提供协方差，因为均值会变
# 这里我们只是为了让pdf函数能用，它的mean参数在实际调用pdf时会被覆盖
# 或者更准确地说，可以不提供mean，直接在pdf调用时指定
# dist = multivariate_normal(cov=observation_covariance_R) # 也可以这样

# --- 2. 定义一个单个观测点 x_single ---
# 假设这是机器人实际接收到的观测 (z_x, z_y)
single_observation = np.array([5.2, 3.1])

# --- 3. 定义多个不同的均值 means_multiple ---
# 这些均值代表的是：如果每个粒子是真实状态，那么我们期望的观测会是什么
# 假设有 N=5 个粒子，每个粒子的状态 (x, y, vx, vy) 不同，
# 通过 C @ state 映射到 2D 期望观测
# 这里的 means_multiple 相当于之前代码中的 expected_observations_from_states
num_particles = 5
expected_observations_for_particles = np.array([
    [5.0, 3.2],  # 粒子1的期望观测
    [5.3, 3.0],  # 粒子2的期望观测
    [4.9, 3.5],  # 粒子3的期望观测
    [5.5, 2.9],  # 粒子4的期望观测
    [5.1, 3.15]  # 粒子5的期望观测
])

print(f"Single observation (x_single) shape: {single_observation.shape}")
print(f"Multiple means (means_multiple) shape: {expected_observations_for_particles.shape}")
print(f"Observation Covariance (R) shape: {observation_covariance_R.shape}\n")

# --- 4. 使用 multivariate_normal.pdf 一次性计算 ---
# 直接调用 pdf 函数，传入单个 x 和多个 mean
# SciPy 会自动将 single_observation 广播，对每个 mean 进行计算
pdf_values_vectorized = multivariate_normal_pdf_vectorized(
    single_observation,
    expected_observations_for_particles, # 传入多个均值
    observation_covariance_R
)

print("--- Vectorized Calculation ---")
print("PDF values (vectorized):", pdf_values_vectorized)
print(f"Shape of vectorized result: {pdf_values_vectorized.shape}\n")

# --- 5. 通过循环单独计算每个情况，进行结果验证 ---
pdf_values_loop = np.zeros(num_particles)
print("--- Loop-based Verification ---")
for i in range(num_particles):
    # 为每个粒子单独创建并计算 PDF
    current_particle_mean = expected_observations_for_particles[i, :]
    
    # 直接创建临时分布对象或再次调用 pdf
    # 这里为了演示，我们直接在每次循环中传入 mean
    pdf_value_i = multivariate_normal.pdf(
        x=single_observation,
        mean=current_particle_mean, # 单个均值
        cov=observation_covariance_R
    )
    pdf_values_loop[i] = pdf_value_i
    print(f"Particle {i+1} (Mean: {current_particle_mean}): PDF = {pdf_value_i}")

print("\nPDF values (loop-based):", pdf_values_loop)

# --- 比较两种方法的结果 ---
print("\n--- Comparison ---")
if np.allclose(pdf_values_vectorized, pdf_values_loop):
    print("Vectorized and loop-based results are identical (within numerical precision).")
else:
    print("WARNING: Results differ!")

Single observation (x_single) shape: (2,)
Multiple means (means_multiple) shape: (5, 2)
Observation Covariance (R) shape: (2, 2)

--- Vectorized Calculation ---
PDF values (vectorized): [1.30642333e+00 5.85498315e+00 5.93115274e-05 2.39279779e-02
 8.51895022e+00]
Shape of vectorized result: (5,)

--- Loop-based Verification ---
Particle 1 (Mean: [5.  3.2]): PDF = 1.3064233284684859
Particle 2 (Mean: [5.3 3. ]): PDF = 5.85498315243193
Particle 3 (Mean: [4.9 3.5]): PDF = 5.931152735254172e-05
Particle 4 (Mean: [5.5 2.9]): PDF = 0.023927977920047094
Particle 5 (Mean: [5.1  3.15]): PDF = 8.518950219522608

PDF values (loop-based): [1.30642333e+00 5.85498315e+00 5.93115274e-05 2.39279779e-02
 8.51895022e+00]

--- Comparison ---
Vectorized and loop-based results are identical (within numerical precision).


In [3]:
import numpy as np
from scipy.stats import multivariate_normal

# --- 1. 定义一个 2D 的多元正态分布 (协方差固定) ---
observation_covariance_R = np.array([
    [0.01, 0.00],
    [0.00, 0.01]
])

# 创建一个 "冻结" 的分布对象，只提供协方差。
# 注意：这里mean设置为np.zeros(2)明确了维度，或者可以完全省略mean参数
# multivariate_normal(cov=observation_covariance_R) 也是可以的
# 但在pdf调用时，我们会覆盖mean参数
dist = multivariate_normal(mean=np.zeros(2), cov=observation_covariance_R)

# --- 2. 定义一个单个观测点 x_single ---
single_observation = np.array([5.2, 3.1])

# --- 3. 定义多个不同的均值 means_multiple ---
num_particles = 5
expected_observations_for_particles = np.array([
    [5.0, 3.2],
    [5.3, 3.0],
    [4.9, 3.5],
    [5.5, 2.9],
    [5.1, 3.15]
])

print(f"Single observation (x_single) shape: {single_observation.shape}")
print(f"Multiple means (means_multiple) shape: {expected_observations_for_particles.shape}")
print(f"Observation Covariance (R) shape: {observation_covariance_R.shape}\n")

# --- 4. 使用 multivariate_normal.pdf 一次性计算 ---
# 这里是关键：
# x 传入单个观测点 (D,)
# mean 传入多个期望观测点 (N, D)
# scipy.stats.multivariate_normal.pdf 内部会正确处理这种广播
# 它会计算 single_observation 相对于每一个 expected_observations_for_particles[i] 的概率密度
pdf_values_vectorized = dist.pdf(
    x=single_observation,
    mean=expected_observations_for_particles # 传入多个均值
)

print("--- Vectorized Calculation ---")
print("PDF values (vectorized):", pdf_values_vectorized)
print(f"Shape of vectorized result: {pdf_values_vectorized.shape}\n")

# --- 5. 通过循环单独计算每个情况，进行结果验证 ---
pdf_values_loop = np.zeros(num_particles)
print("--- Loop-based Verification ---")
for i in range(num_particles):
    current_particle_mean = expected_observations_for_particles[i, :]
    
    # 每次循环都针对当前粒子的均值进行计算
    pdf_value_i = dist.pdf(
        x=single_observation,
        mean=current_particle_mean, # 单个均值
    )
    pdf_values_loop[i] = pdf_value_i
    print(f"Particle {i+1} (Mean: {current_particle_mean}): PDF = {pdf_value_i}")

print("\nPDF values (loop-based):", pdf_values_loop)

# --- 比较两种方法的结果 ---
print("\n--- Comparison ---")
if np.allclose(pdf_values_vectorized, pdf_values_loop):
    print("Vectorized and loop-based results are identical (within numerical precision).")
else:
    print("WARNING: Results differ!")

Single observation (x_single) shape: (2,)
Multiple means (means_multiple) shape: (5, 2)
Observation Covariance (R) shape: (2, 2)



TypeError: multivariate_normal_frozen.pdf() got an unexpected keyword argument 'mean'